In [1]:
# Process OBS so it's ready for comparison

In [2]:
import pandas as pd

# Load observed on-board survey (OBS) trip records
# obs = pd.read_csv(r'R:\FastTrips\CHTS & OBS ft_output\OBS_ft_output_w_stops.csv')

# obs.fillna('',inplace=True)
# obs['person_id'] = obs['person_id'].apply(lambda x: x.split("_")[-1]).astype('str')
# obs['A_id_num'] = obs['A_id_num'].replace("",0)
# obs['A_id_num'] = obs['A_id_num'].astype('int')

# # trip_list_id_num is wrong on current input - replace with correct value from input
# obs = obs.drop('trip_list_id_num',axis=1)    
# obsinput = pd.read_csv(r'J:\Projects\FasTrips\obs\input\1.0\OBS_fasttrips_demand_v1.0\trip_list.txt')
# obsinput['person_id'] = obsinput['person_id'].astype('int').astype('str')

# obs = pd.merge(obs, obsinput[['person_id','trip_list_id_num']], on='person_id')
# obs.to_csv(r'R:\FastTrips\CHTS & OBS ft_output\OBS_ft_output_w_stops_BN.csv')

In [272]:
# Read latest OBS data
df = pd.read_csv(r'R:\FastTrips\CHTS & OBS ft_output\OBS_ft_output_w_stops.csv')

In [273]:
df['person_id'] = df['person_id'].apply(lambda x: x.split("_")[-1]).astype('str')

In [274]:
# Fill NaN type with empty string
for field in ['A_id_num','B_id_num','A_id','B_id']:
    df[field].fillna("",inplace=True)

In [275]:
# trip_list_id_num is wrong on current input - replace with correct value from input
df = df.drop('trip_list_id_num',axis=1)

# Read in an Fast Trips input format
obsinput = pd.read_csv('../data/obs/trip_list.txt')
obsinput['person_id'] = obsinput['person_id'].astype('int').astype('str')

In [276]:
df = pd.merge(df, obsinput[['person_id','trip_list_id_num']], on='person_id')

In [277]:
# Add the proper transit fields!
obs_to_gtfs = pd.read_csv('../data/obs/obs_to_gtfs_v1.8.csv')
agency_lookup = pd.read_csv('../data/obs/obs_agency_lookup.csv')

obs_to_gtfs.fillna("",inplace=True)
obs_to_gtfs.drop_duplicates(inplace=True)

In [278]:
# Get a unique lookup of OBS to GTFS agency names
agency_list = obs_to_gtfs.drop_duplicates(subset=['OBS_agency'])[['OBS_agency','GTFS1.8_agency']].reset_index(drop=True)

# drop any empty OBS rows 
agency_list = agency_list[agency_list['OBS_agency'] != ""]

In [286]:
# Join the OBS agency name
df = pd.merge(df,agency_lookup,on='route_id',how='left')

In [287]:
# Join the GTFS agency name to the OBS data
df = pd.merge(df, agency_list, left_on='agency', right_on='OBS_agency', how='left')

In [289]:
# Replace the OBS 
df['agency'] = df['GTFS1.8_agency']
df.drop(['OBS_agency','GTFS1.8_agency'],axis=1,inplace=True)


In [291]:
# See if we can extract any route IDs from here based on that route_id field

In [296]:
rows_with_route_id = df[-df['route_id'].isnull()]
rows_without_route_id = df[df['route_id'].isnull()]

In [297]:
len(rows_with_route_id) + len(rows_without_route_id) == len(df)

True

In [299]:
newdf = pd.merge(rows_with_route_id, obs_to_gtfs, left_on='route_id', right_on='OBS_route_id')

In [314]:
output = pd.concat([newdf,rows_without_route_id])

In [316]:
output.sort()

,A_id,A_id_num,B_id,B_id_num,GTFS1.8_agency,GTFS1.8_route_id,OBS_agency,OBS_route_id,Unnamed: 4,agency,linkmode,linknum,mode,person_id,route_id,trip_list_id_num
0,,,,,caltrain,,Caltrain,Caltrain_Burlingame California Ave,,caltrain,transit,1,commuter_rail,100,Caltrain_Burlingame California Ave,28
0,1258,1258,,,NaN,NaN,NaN,NaN,NaN,NaN,access,0,walk_access,100,NaN,28
1,,,,,caltrain,,Caltrain,Caltrain_Burlingame California Ave,,caltrain,transit,3,commuter_rail,1049,Caltrain_Burlingame California Ave,469
2,,,,,caltrain,,Caltrain,Caltrain_Burlingame California Ave,,caltrain,transit,1,commuter_rail,3106,Caltrain_Burlingame California Ave,1833
2,,,1358,1358,NaN,NaN,NaN,NaN,NaN,NaN,egress,2,walk_egress,100,NaN,28
3,,,,,caltrain,,Caltrain,Caltrain_Burlingame California Ave,,caltrain,transit,1,commuter_rail,748,Caltrain_Burlingame California Ave,298
3,930,930,24893,24893,NaN,NaN,NaN,NaN,NaN,NaN,access,0,walk_access,1000,NaN,431
4,24893,24893,,,sf_muni,,MUNI,MUNI,,sf_muni,transit,1,local_bus,1000,MUNI,431
4,930,930,24893,24893,NaN,NaN,NaN,NaN,NaN,NaN,access,0,walk_access,1000,NaN,432
5,24893,24893,,,sf_muni,,MUNI,MUNI,,sf_muni,transit,1,local_bus,1000,MUNI,432


In [305]:
len(newdf.drop_duplicates())

34277

In [307]:
len(newdf)

34278

In [306]:
len(rows_with_route_id)

32815

In [301]:
len(newdf)

34278

In [207]:
print len(set(obs_routes['OBS_route_id'].values).intersection(df['route_id']))

1990


In [208]:
len(obs_routes.groupby('OBS_route_id').count().index)

3485

In [209]:
len(df.groupby('route_id').count().index)

2099

In [210]:
# df[df['route_id'] == 'AC Transit_1']
# df

In [211]:
df['OBS_route_id'] = df['route_id']
newdf = pd.merge(df,obs_routes,on='OBS_route_id',how='left')


In [212]:
newdf

,person_id,linkmode,A_id_num,B_id_num,A_id,B_id,linknum,mode,route_id,OBS_route_id,trip_list_id_num,OBS_agency,GTFS1.8_route_id,GTFS1.8_agency,Unnamed: 4
0,100,access,1258,,1258,,0,walk_access,,,28,NaN,NaN,NaN,NaN
1,100,transit,,,,,1,commuter_rail,Caltrain_Burlingame California Ave,Caltrain_Burlingame California Ave,28,Caltrain,,caltrain,
2,100,egress,,1358,,1358,2,walk_egress,,,28,NaN,NaN,NaN,NaN
3,1000,access,930,24893,930,24893,0,walk_access,,,431,NaN,NaN,NaN,NaN
4,1000,access,930,24893,930,24893,0,walk_access,,,432,NaN,NaN,NaN,NaN
5,1000,access,930,24893,930,24893,0,walk_access,,,431,NaN,NaN,NaN,NaN
6,1000,access,930,24893,930,24893,0,walk_access,,,432,NaN,NaN,NaN,NaN
7,1000,transit,24893,,24893,,1,local_bus,MUNI,MUNI,431,MUNI,,sf_muni,
8,1000,transit,24893,,24893,,1,local_bus,MUNI,MUNI,432,MUNI,,sf_muni,
9,1000,transit,24893,,24893,,1,local_bus,MUNI,MUNI,431,MUNI,,sf_muni,


In [213]:
len(newdf)

107158

In [200]:
newdf.groupby('GTFS1.8_agency').count()

,person_id,linkmode,A_id_num,B_id_num,A_id,B_id,linknum,mode,route_id,OBS_route_id,trip_list_id_num,OBS_agency,GTFS1.8_route_id,Unnamed: 4
GTFS1.8_agency,,,,,,,,,,,,,,
EXTERNAL,1,1,1,1,1,1,1,1,1,1,1,1,0,0
UNMODELED,386,386,386,386,386,386,386,386,386,386,386,386,0,0
ac_transit,8329,8329,8329,8329,8329,8329,8329,8329,8329,8329,8329,8329,5834,1895
ace,6,6,6,6,6,6,6,6,6,6,6,6,6,6
amtrak,21,21,21,21,21,21,21,21,21,21,21,21,21,21
bart,13171,13171,13171,13171,13171,13171,13171,13171,13171,13171,13171,13171,0,0
caltrain,2959,2959,2959,2959,2959,2959,2959,2959,2959,2959,2959,2959,0,0
cccta,566,566,566,566,566,566,566,566,566,566,566,566,0,0
ferry,521,521,521,521,521,521,521,521,521,521,521,521,476,0


In [142]:
newdf.groupby('GTFS1.8_route_id').count()

,person_id,linkmode,A_id_num,B_id_num,A_id,B_id,linknum,mode,route_id,trip_list_id_num,OBS_route_id,OBS_agency,GTFS1.8_agency,Unnamed: 4
GTFS1.8_route_id,,,,,,,,,,,,,,
,21421,21421,21421,21421,21421,21421,21421,21421,21421,21421,21421,21421,21421,21421
ac_transit_11,83,83,83,83,83,83,83,83,83,83,83,83,83,83
ac_transit_12,119,119,119,119,119,119,119,119,119,119,119,119,119,119
ac_transit_13,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989
ac_transit_13A,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989
ac_transit_14,171,171,171,171,171,171,171,171,171,171,171,171,171,171
ac_transit_210,31,31,31,31,31,31,31,31,31,31,31,31,31,31
ac_transit_211,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989,58989
ac_transit_212,23,23,23,23,23,23,23,23,23,23,23,23,23,23


In [143]:
newdf

,person_id,linkmode,A_id_num,B_id_num,A_id,B_id,linknum,mode,route_id,trip_list_id_num,OBS_route_id,OBS_agency,GTFS1.8_route_id,GTFS1.8_agency,Unnamed: 4
0,100,access,1258,,1258,,0,walk_access,,28,,AC Transit,ac_transit_13,ac_transit,
1,100,access,1258,,1258,,0,walk_access,,28,,AC Transit,ac_transit_13A,ac_transit,
2,100,access,1258,,1258,,0,walk_access,,28,,AC Transit,ac_transit_211,ac_transit,
3,100,access,1258,,1258,,0,walk_access,,28,,AC Transit,ac_transit_213,ac_transit,
4,100,access,1258,,1258,,0,walk_access,,28,,AC Transit,ac_transit_214,ac_transit,
5,100,access,1258,,1258,,0,walk_access,,28,,AC Transit,ac_transit_218,ac_transit,
6,100,access,1258,,1258,,0,walk_access,,28,,AC Transit,ac_transit_231,ac_transit,
7,100,access,1258,,1258,,0,walk_access,,28,,AC Transit,ac_transit_235,ac_transit,
8,100,access,1258,,1258,,0,walk_access,,28,,AC Transit,ac_transit_41,ac_transit,
9,100,access,1258,,1258,,0,walk_access,,28,,AC Transit,ac_transit_43.1,ac_transit,


In [32]:
df.to_csv(r'..\data\obs_chosenpaths_links.csv', index=False)

,person_id,linkmode,A_id_num,B_id_num,A_id,B_id,linknum,mode,route_id,trip_list_id_num
0,100,access,1258,,1258,,0,walk_access,NaN,28
1,100,transit,,,,,1,commuter_rail,Caltrain_Burlingame California Ave,28
2,100,egress,,1358,,1358,2,walk_egress,NaN,28
3,1000,access,930,24893,930,24893,0,walk_access,NaN,431
4,1000,access,930,24893,930,24893,0,walk_access,NaN,432
5,1000,transit,24893,,24893,,1,local_bus,MUNI,431
6,1000,transit,24893,,24893,,1,local_bus,MUNI,432
7,1000,transfer,,23820,,23820,2,transfer,NaN,431
8,1000,transfer,,23820,,23820,2,transfer,NaN,432
9,1000,transit,23820,,23820,,3,commuter_rail,Caltrain_San Francisco Burlingame,431
